In [201]:
from random import *
from utils import accepts_tuple_arg, add_tuples
from copy import deepcopy
import time
# different pieces/players
WHITE = 0
BLACK = 1
def getOtherColor(color):
    if color == WHITE:
        return BLACK
    return WHITE
def number_of_pieces(color, board):
    """
    :param color: which color's pieces you're counting
    :param alive_pieces: array with all the current alive pieces on the board
    :return: number of alive pieces of a certain color

    Finds number of pieces left on the board of a certain color.
    """
    return board.pieces_left(color)

def offensive1(player, board):
    """
    :param player: color of the player whose turn it is
    :param board: current state of the board
    :return: score for each move (higher is better)

    The more pieces your opponent has, the lower the score will be.
    """
    color = getOtherColor(player)
    num_pieces = number_of_pieces(color, board)

    return 5 * (30 - num_pieces) + random()

def defensive1(player, board):
    """
    :param player: color of the player whose turn it is
    :param board: current state of the board
    :return: score for each move (higher is better)

    The more pieces you opponent have, the higher the score will be.
    """

    num_pieces = number_of_pieces(player, board)   # find number of your pieces

    return 2 * num_pieces + random()

def offensive2(player, board):
    """
    Minimize enemy undefended pieces
    """
    color = getOtherColor(player)
    num_pieces = number_of_pieces(color, board)
    pieces_to_attack = board.getAttackedPieces(color)
    return 5 * (30 - num_pieces) + len(pieces_to_attack) + random()
    
def defensive2(player, board):
    """
    Give leaving pieces defended extra weight
    """
    numPieces = board.pieces_left(player)
    #undefendedPieces = [piece for piece in board.getAttackedPieces(player) if not piece.isDefended(board)]
    defendedPieces = board.getDefendedPieces(player)
    attackedPieces = board.getAttackedPieces(player)
    return numPieces + .1 * len(defendedPieces) - .5*len(attackedPieces)

In [184]:
# different pieces/players
WHITE = 0
BLACK = 1
ATTACK_SQUARES = [[(-1,1),(1,1)],[(-1,-1),(1,-1)]]
MOVE_SQUARES = [(0,1),(0,-1)]

class Piece:

    def __init__(self):
        self.color = None
        self.x = None
        self.y = None
    @accepts_tuple_arg
    def move(self, x, y, board):
        if board.getCoord(x,y) != None:
            board.getCoord(x,y).remove(board)
        board.setCoord(x,y, self)

        board.setCoord(self.x,self.y, None)
        self.x = x
        self.y = y
        return True

    @accepts_tuple_arg
    def canAttackSquare(self, board, x, y):
        piece = board.getCoord(x,y)
        if piece == None:
            return False
        if piece.color == self.color:
            return False
        return True
    
    def getLegalMovements(self, board):
        movements = [add_tuples((self.x,self.y),MOVE_SQUARES[self.color])]
        if board.getCoord(movements[0]) != None:
            movements = []
        return movements
    def getLegalAttacks(self, board):
        attacks = [add_tuples((self.x,self.y),attack_square) for attack_square in ATTACK_SQUARES[self.color]]
        attacks = list(filter(lambda coord: self.canAttackSquare(board, coord), attacks))
        return attacks
    def getLegalMoves(self, board):
        return self.getLegalMovements(board)+self.getLegalAttacks(board)
    #If this piece can attack another piece, it is attackable
    def is_attackable(self, board):
        return len(self.getLegalAttacks(board))>0
    
    def isDefended(self, board):
        orig = self.color
        self.color = self.getOtherColor()
        attackable = self.is_attackable(board)
        self.color = orig
        return attackable
    def remove(self, board):
        board.alive_pieces.remove(self)
        board.pieces[self.color].remove(self)
    def copy(self, newBoard = None):
        newPiece = Piece()
        newPiece.x = self.x
        newPiece.y = self.y
        newPiece.color = self.color
        if newBoard != None:
            newBoard.setCoord(self.x, self.y, newPiece)
        return newPiece
    def __str__(self):
        string = ""
        if self.color==WHITE:
            string="W"
        else:
            string="B"
        string+="("+str(self.x)+","+str(self.y)+")"
        return string
    def getOtherColor(self):
        if self.color == WHITE:
            return BLACK
        else:
            return WHITE
    def unable_to_attack(self, board):
        return length(self.getLegalAttacks(board))==0


In [183]:
class Board:
    def __init__(self):
        self.board = [[None for i in range(8)] for i in range(8)]  # game board
        self.alive_pieces = []  # keeps track of alive pieces in current game
        self.pieces = [[],[]] #seperates the pieces by color
    @accepts_tuple_arg
    def getCoord(self, x, y):
        if x < 0 or x > 7 or y < 0 or y > 7: 
            return None
        return self.board[x][y]
    @accepts_tuple_arg
    def setCoord(self, x, y, piece):
        try:
            self.board[x][y] = piece
        except IndexError:
            print(self)
            print(piece)
            print(x,y)
            raise IndexError("Error")
            
        
    def getEquivalentPiece(self, piece):
        return self.getCoord(piece.x, piece.y)
    def set_board(self):
        """sets board up in beginning of the game"""
        for i in range(0, 8):
            for j in range(0, 8):
                if i == 0 or i == 1:
                    new_piece = Piece()
                    new_piece.color = WHITE
                    new_piece.x = j
                    new_piece.y = i
                    self.setCoord(j, i , new_piece)
                    self.alive_pieces.append(new_piece)
                    self.pieces[0].append(new_piece)
                elif i == 6 or i == 7:
                    new_piece = Piece()
                    new_piece.color = BLACK
                    new_piece.x = j
                    new_piece.y = i
                    self.setCoord(j, i, new_piece)
                    self.alive_pieces.append(new_piece)
                    self.pieces[1].append(new_piece)

    def copy(self):
        """copies current state of the board to a new board"""
        new_board = Board()
        for piece in self.alive_pieces:
            newPiece = piece.copy(new_board)
            new_board.alive_pieces.append(newPiece)
            new_board.pieces[piece.color].append(newPiece)
        return new_board

    def print_board(self):
        """prints board state"""
        curr_line = "|"
        for i in range(0, 8):
            for j in range(0, 8):
                if self.board[i][j] is None:
                    curr_line += ' |'
                elif self.board[i][j].color == WHITE:
                    curr_line += 'W|'
                elif self.board[i][j].color == BLACK:
                    curr_line += 'B|'
                else:
                    curr_line += ' '
            print(curr_line)
            
            curr_line = "|"
    def __str__(self):
        curr_line=""
        for j in reversed(range(0,8)):
            curr_line+="|"
            for i in range(0,8):
                if self.getCoord(i,j) is None:
                    curr_line += ' |'
                elif self.getCoord(i,j).color == WHITE:
                    curr_line += 'W|'
                elif self.getCoord(i,j).color == BLACK:
                    curr_line += 'B|'
                else:
                    curr_line += ' '
            curr_line+="\n"
        return curr_line
                
    def pieces_left(self, color):
        return len(self.pieces[color])
    def getLegalMoves(self, color):
        legal_moves = []
        for piece in self.pieces[color]:
            legal_moves += [(piece, move) for move in piece.getLegalMoves(self)]
        return legal_moves
    def getAttackedPieces(self, color):
        return [ piece for piece in self.pieces[color] if piece.is_attackable(board)]
    def getDefendedPieces(self, color):
        return [ piece for piece in self.pieces[color] if piece.isDefended(board)]
    def isGameOver(self):
        for i in range(8):
            if self.getCoord(i,0) != None and self.getCoord(i,0).color == BLACK:
                return True, BLACK
            if self.getCoord(i,7) != None and self.getCoord(i,7).color == WHITE:
                return True, WHITE
        if self.pieces_left(WHITE) == 0:
            return True, BLACK
        if self.pieces_left(BLACK) == 0:
            return True, WHITE
        return False, None
board = Board()
board.set_board()
print(board)
print(board.getDefendedPieces(WHITE))


|B|B|B|B|B|B|B|B|
|B|B|B|B|B|B|B|B|
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
|W|W|W|W|W|W|W|W|
|W|W|W|W|W|W|W|W|

[<__main__.Piece object at 0x7fe9a839ee80>, <__main__.Piece object at 0x7fe9a839e828>, <__main__.Piece object at 0x7fe9a839ef60>, <__main__.Piece object at 0x7fe9a839e8d0>, <__main__.Piece object at 0x7fe9a839efd0>, <__main__.Piece object at 0x7fe9a839e048>, <__main__.Piece object at 0x7fe9a839e898>, <__main__.Piece object at 0x7fe9a839ed30>]


In [186]:
def minimax(board, color, heuristic, depth, useAlphaBeta = False):
    alpha = -float('inf')
    beta = float('inf')
    node = MaxNode(board, color, heuristic, depth, alpha, beta, useAlphaBeta)
    
    return node.bestBoard, node.nodesExpanded
    
    
class MiniMaxNode:
    def __init__(self, board, color, heuristic, depth):
        self.board = board
        self.color = color
        self.heuristic = heuristic
        self.depth = depth
        self.nodesExpanded = 1
        self.children = []
        
    def getLegalBoards(self, color):
        legalMoves = self.board.getLegalMoves(color)
        legalBoards = []
        for piece, move in legalMoves:
            newBoard = self.board.copy()
            newPiece = newBoard.getEquivalentPiece(piece)
            newPiece.move(move, newBoard)
            legalBoards+=[newBoard]
            self.piece = piece
            self.move = move
        return legalBoards
    def calculateUtility(self):
        self.score = self.heuristic(self.color, self.board)
    
class MaxNode(MiniMaxNode):
    def __init__(self, board, color, heuristic, depth, alpha=0, beta=0, useAlphaBeta =False):
        MiniMaxNode.__init__(self,board, color, heuristic, depth)
        game_over, winner = board.isGameOver()
        if game_over:
            if winner == color:
                self.score = 999
                return
            else:
                self.score = -999
                return
        if depth==0:
            self.calculateUtility()
            return 
        self.score = -9999
        for board in self.getLegalBoards(color):
            child = MinNode(board, color, heuristic, depth-1, alpha, beta, useAlphaBeta)
            self.nodesExpanded += child.nodesExpanded
            if child.score > self.score:
                self.score = child.score
                self.bestBoard = board
            alpha = max(alpha, self.score)
            if useAlphaBeta and beta <= alpha:
                return
        return
class MinNode(MiniMaxNode):
    def __init__(self, board, color, heuristic, depth, alpha=0, beta=0, useAlphaBeta = False):
        MiniMaxNode.__init__(self,board, color, heuristic, depth)
        game_over, winner = board.isGameOver()
        if game_over:
            if winner == color:
                self.score = 999
                return
            else:
                self.score = -999
                return
        if depth==0:
            self.calculateUtility()
            return 
        self.score = -9999
        for board in self.getLegalBoards(getOtherColor(color)):
            child = MaxNode(board, color, heuristic, depth-1, alpha, beta, useAlphaBeta)
            self.nodesExpanded += child.nodesExpanded
            if child.score > self.score:
                self.score = child.score
                self.bestBoard = board
            beta = min(beta, self.score)
            if useAlphaBeta and beta <= alpha:
                return
        return

board = Board()
board.set_board()
a, nodes = minimax(board, WHITE, offensive1, 3, True)
print(a)
print(nodes)


|B|B|B|B|B|B|B|B|
|B|B|B|B|B|B|B|B|
| | | | | | | | |
| | | | | | | | |
| | | | | | | | |
| | | | |W| | | |
|W|W|W|W| |W|W|W|
|W|W|W|W|W|W|W|W|

261


In [209]:
def game_play(debug = True):
    """runs through game"""
    game_board = Board()
    game_board.set_board()
    turn = WHITE  # which player's turn it is
    num_moves = 0
    time_per_move = []
    nodes_per_move = []
    black_nodes_expanded = 0
    white_nodes_expanded = 0

    while 1:  # game loop
        start_time = time.time()
        # call heuristic functions

        if turn == WHITE:
            game_board, nodes_expanded = minimax(game_board, turn, offensive2, 5, True)
        else:
            game_board, nodes_expanded = minimax(game_board, turn, defensive2, 5, True)

        nodes_per_move.append(nodes_expanded)
        if turn == WHITE:
            white_nodes_expanded += nodes_expanded
        else:
            black_nodes_expanded += nodes_expanded
        end_time = time.time()
        time_per_move.append(end_time-start_time)
        num_moves += 1
        if debug:
            print(game_board)
            print("\n")
            print('----------------------------')
            print("\n")

        # breaks from loop when game is over
        gameover, winner = game_board.isGameOver()
        if gameover:
            if winner == WHITE:
                player = "white"
            else:
                player = "black"
            print(game_board)
            print("Winner is: " + player)
            print("Number of moves: ", num_moves)
            print("Average time per move: " + str(sum(time_per_move) / len(time_per_move)))
            print("Average nodes per move: " + str(sum(nodes_per_move) / len(nodes_per_move)))
            print("Nodes expanded by white: " + str(white_nodes_expanded))
            print("Nodes expanded by black: " + str(black_nodes_expanded))
            print("Pieces captured by white: ", 16-game_board.pieces_left(BLACK))
            print("Pieces captured by black: ", 16-game_board.pieces_left(WHITE))
            break

        # switches players turn
        if turn == WHITE:
            turn = BLACK
        else:
            turn = WHITE

if __name__ == '__main__':
    game_play(False)

| | |B| | | | |B|
|W|B|W|B|B|B|B| |
| | |W| |B|B|B|B|
| |W| | | | | | |
|B| | | | | | | |
|W| | | | | | | |
|W| | |W| |W|W|W|
| | | |B|W|W|W|W|

Winner is: black
Number of moves:  42
Average time per move: 1.9742557037444342
Average nodes per move: 3150.9523809523807
Nodes expanded by white: 49577
Nodes expanded by black: 82763
Pieces captured by white:  3
Pieces captured by black:  2
